# Trigger ArcticDEM Download 
### Notebook to download
1. detrended and
2. rescaled Arctic DEM data

* run with gee environment: aicore has geopandas/pyproj issues

In [ ]:
import ee
import os
import geopandas as gpd
import glob
import eeconvert
%config Completer.use_jedi = False # dummy line to "fix" buggy autocomplete

In [ ]:
ee.Initialize()

### Settings

#### Set path to diretory with boundary files

In [ ]:
REGION_FILE = r''

#### Read all files
* change pattern to '/*.shp' for shapefiles or keep for geojson

In [ ]:
flist = glob.glob(REGION_FILE + '/*')

In [ ]:
flist

### Functions

In [ ]:
def get_ArcticDEM_slope():
    dem = ee.Image("UMN/PGC/ArcticDEM/V3/2m_mosaic")
    slope = ee.Terrain.slope(dem)
    return slope

In [ ]:
def get_ArcticDEM_rel_el(kernel_size=300, offset=50, factor=300):
    dem = ee.Image("UMN/PGC/ArcticDEM/V3/2m_mosaic")
    conv = dem.convolve(ee.Kernel.circle(kernel_size, 'meters'))
    diff = (dem.subtract(conv).add(ee.Image.constant(offset)).multiply(ee.Image.constant(factor)).toInt16())
    return diff

In [ ]:
def download_ee(filepath, ee_layer, GDrive_dir='AI-CORE/slope', scale=2, buffersize=5000, suffix='_slope'):
    
    name = os.path.basename(filepath).split('.')[0] + suffix
    gdf = gpd.read_file(filepath).to_crs(crs='EPSG:4326')
    
    ee_fc = eeconvert.gdfToFc(gdf)
    buffered_geom = ee_fc.geometry().buffer(5000)
    
    dl = ee.batch.Export.image.toDrive(
        image=ee_layer,
        description=name,
        folder=GDrive_dir,
        fileNamePrefix=name,
        dimensions=None,
        region=buffered_geom,
        scale=2,
        crs=None,
        crsTransform=None,
        maxPixels=1e13,
        shardSize=None,
        fileDimensions=None,
        skipEmptyTiles=None,
        fileFormat=None,
        formatOptions=None)
    
    dl.start()
    
    return dl

#### Access GEE data layer

In [ ]:
slope = get_ArcticDEM_slope()

In [ ]:
rel_el_100 = get_ArcticDEM_rel_el(kernel_size=100)

### Loop over datasets
#### Slope

#### Run full loop over slope 

In [ ]:
tasks = [download_ee(f, slope, GDrive_dir='AI-CORE/slope', suffix='_slope') for f in flist[:]]

In [ ]:
tasks[0].status()

In [ ]:
print('Running Tasks:')
[t.status()['description'] for t in tasks if t.status()['state'] == 'RUNNING']

In [ ]:
print('Completed Tasks:')
[t.status()['description'] for t in tasks if t.status()['state'] == 'COMPLETED']

### Loop over datasets
#### Elevation

#### Run full loop over elevation

In [ ]:
tasks_el = [download_ee(f, rel_el_100, GDrive_dir='AI-CORE/elevation', suffix='_rel_el_100_scaled') for f in flist[:]]

In [ ]:
print('Running Tasks:')
[t.status()['description'] for t in tasks_el if t.status()['state'] == 'RUNNING']

In [ ]:
print('Completed Tasks:')
[t.status()['description'] for t in tasks_el if t.status()['state'] == 'COMPLETED']